### Yolo v3 model import
Run this block to import necessary libraries used by the code, and also the yolo v3 model.  
Two models can be selected, yolov3.weights and yolov3-tiny.weights, which have different performance.  
Select one of them by commenting the other line out.

In [8]:
import cv2
import numpy as np
import time

# Load Yolo
net = cv2.dnn.readNet("weights/yolov3-tiny.weights", "cfg/yolov3-tiny.cfg")
# net = cv2.dnn.readNet("weights/yolov3.weights", "cfg/yolov3.cfg")
classes = []
with open("data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

### Color detection implementation
This is the implementation of color detection.  
Input is an RGB image containing a traffic light.  
Output is a string indicating the color name of the traffic light in the image

In [9]:
# color detection
def detect_color(image):
    # Convert the imageFrame in 
    # BGR(RGB color space) to 
    # HSV(hue-saturation-value)
    # color space
    hsvFrame = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  
    # Set range for red color and 
    # define mask
    red_lower = np.array([136, 87, 111], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)
  
    # Set range for green color and 
    # define mask
    green_lower = np.array([25, 52, 72], np.uint8)
    green_upper = np.array([102, 255, 255], np.uint8)
    green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)

    # Set range for yellow color and 
    # define mask
    yellow_lower = np.array([22, 93, 0], np.uint8)
    yellow_upper = np.array([45, 255, 255], np.uint8)
    yellow_mask = cv2.inRange(hsvFrame, yellow_lower, yellow_upper)
  
    # Morphological Transform, Dilation
    # for each color and bitwise_and operator
    # between imageFrame and mask determines
    # to detect only that particular color
    kernal = np.ones((5, 5), "uint8")
      
    # For red color
    red_mask = cv2.dilate(red_mask, kernal)
      
    # For green color
    green_mask = cv2.dilate(green_mask, kernal)
    
    # For yellow color
    yellow_mask = cv2.dilate(yellow_mask, kernal)

     # Creating contour to track red color
    contours, hierarchy = cv2.findContours(red_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
      
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            return 'Red'
  
    # Creating contour to track green color
    contours, hierarchy = cv2.findContours(green_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
      
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            return 'Green'

    # Creating contour to track yellow color
    contours, hierarchy = cv2.findContours(yellow_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            return 'Yellow'
    
    return 'None'


### Main function
The main function starts here. A video is loaded first, different videos can be tested by changeing the file name in cv2.VideoCapture().
The information of video is retrieved and each frame is detected with yolo first and the traffic light region is preocessed with the color detection function. Then the light color of each frame can be obtained and displayed in real time. After the last frame is processed, the output image of each frame are put together into a complete video which is stored locally.

In [10]:
cap = cv2.VideoCapture('test1.mp4')
# cap = cv2.VideoCapture(0)

# get the properties of the input video
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Define the codec and create VideoWriter object to store video
output = cv2.VideoWriter('output.mp4',fourcc, fps, size)

starting_time = time.time()
frame_id = 0
while True:
    ret, frame = cap.read()
    if ret:
        frame_id += 1
        height, width, channels = frame.shape

        # Detecting objects
        mean=(0, 0, 0)
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), mean, swapRB=True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # plotting the objects detected
        class_ids = []
        confidences = []
        boxes = []
        light_colors = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                # check only traffic light class (class 9)
                if (class_id != 9):
                    continue
                confidence = scores[class_id]
                if confidence > 0.3:
                    # when confidence over threshold object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    if (x >= 0 and y >= 0):
                        # get the target region with traffic lights
                        target_img = frame[y:y+h,x:x+w]

                    # detect color in the region
                    if not (target_img is None):
                        light_color = detect_color(target_img)
    
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
                    light_colors.append(light_color)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

        # display boxes and texts of each traffic light detected
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                light_color = light_colors[i]
                color = (0, 0, 0)
                if (light_color == 'Red'):
                    color = (0, 0, 255)
                if (light_color == 'Yellow'):
                    color = (0, 255, 255)
                if (light_color == 'Green'):
                    color = (0, 255, 0)
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, light_color + str(round(confidence, 2)), (x+w+10, y+30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        elapsed_time = time.time() - starting_time
        fps = frame_id / elapsed_time
        cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        cv2.imshow("Image", frame)
        output.write(frame)
        key = cv2.waitKey(1)
        if key == 27:
            break
    else:
        break

cap.release()
# close the videowriter
output.release()
cv2.destroyAllWindows()